# Rebalance Zonation

This rebalances the zonation results to correct for the removal of urban and reservoirs.

Originally I had this code in with the code to combine all the zonation runs, but since we do a lot of testing of zonation results, it works better to have them seperate.

In [438]:
import os, fnmatch
import arcpy

In [439]:
import time
start = time.time()

In [440]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)

# this is a working directory for intermediate files
OutWorkspace = r"D:\SE_Blueprint_2023\7_CombineZonation\1_RebalanceZonation.gdb"

In [441]:
# define final outputs
OutTif = r"D:\SE_Blueprint_2023\7_CombineZonation\Z_"

In [442]:
# define rasters used for cell size, extent, and snapping
SEraster= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [443]:
# define additional inputs
# this is the mask file that is used to calculate how much area was left out of each subregion
Z= r"D:\SE_Blueprint_2023\3_Edges\2022SubregionsAndEdgesForWorkshops\West_Gulf_Coastal_Plain_R.tif"
# this is the zonation output that we are rebalancing
Zout= r"D:\SE_Blueprint_2023\6_ZonationRuns\Zonation\West_Gulf_Coastal_Plain_2023\West_Gulf_Coastal_Plain_2023.CAZ_E.rank.compressed.tif"

### Start Analysis

In [444]:
# Set the workspace where outputs go
arcpy.env.workspace = OutWorkspace

In [445]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\7_CombineZonation\1_RebalanceZonation.gdb


## Get Name of Input

In [446]:
# get name of input
outName = os.path.basename(Zout)
print(outName)
outName2 = outName[:-26]
print(outName2)
test = "Stat_" + outName2
print(test)

West_Gulf_Coastal_Plain_2023.CAZ_E.rank.compressed.tif
West_Gulf_Coastal_Plain_2023
Stat_West_Gulf_Coastal_Plain_2023


### Find minimum stretch value 

In [447]:
# calculate statistics for 0/1 raster. 
# This is creating a new table. The table contains statistics for the 0/1 versions of the rasters used as a mask for
# zonation. We need to rescale the zonation outputs because we have removed urban and reservoirs from each subregion. 
# We need to shift the zonation results up to account for all the areas that are really scoring poorly, due to being 
# urban or a reservoir. We calculate the sum of the count field. This is the total number of pixels in the subregion, 
# both 0 (urban or reservoir in the subregion) and 1 (area where zonation was allowed to prioritize in the subrgion).
# Then we find the minumum from the count field. That is the smallest number in the count field, and based on how these
# data were made, we know that is always the number of 0s (there are no subregions with more urban/reservoir than other
# areas). 
arcpy.analysis.Statistics(Z, outName2 + "_Stat", "Count SUM;Count MIN", None)
# use statistics to calcuate the minimum value in the rescale. We take the miniumum value in the count field (the number 
# of 0 pixels) divided by the sum of the count field (total number of pixels 0 and 1). This is the percent of the 
# subregion with 0 values. Then we take it times 100. This will be used as the lower value in our linear rescale function.
arcpy.management.CalculateField(outName2 + "_Stat", "MinValueInRescale", "(!MIN_Count! / !SUM_Count!) * 100", "PYTHON3", '', "DOUBLE")

<Result 'D:\\SE_Blueprint_2023\\7_CombineZonation\\1_RebalanceZonation.gdb\\West_Gulf_Coastal_Plain_2023_Stat'>

### Bring in zonation results

In [448]:
# now I need to take the value from the "MinValueInRescale" field and use it as the "from scale" in my linear rescale
# how do I call up the value from this table in python?
with arcpy.da.SearchCursor(outName2 + "_Stat", 'MinValueInRescale') as cursor:
    for row in cursor:
        zmin = (u'{0}'.format(row[0]))
        print(zmin)

3.7913406483401446


In [449]:
# now I need to take the value from the "MinValueInRescale" field and use it as the "from scale" in my linear rescale
# how do I call up the value from this table in python?
out_raster = arcpy.sa.RescaleByFunction(Zout, "LINEAR # # # # # #", zmin, 100); out_raster.save(outName2 + "_LinearRescale")

In [450]:
# now I need to assign 0 to all the areas that are were reservoirs or urban in that rescaled zonation result. Basically,
# I'm slipping all those areas under the stack of the zonation results, they are all assumed to be worse than any of the 
# areas that we let zonation prioritize.
out_raster = arcpy.sa.Con(Z, Z, outName2 + "_LinearRescale", "Value = 0"); out_raster.save(outName2 + "UrbRes")

In [451]:
# convert to an integer
out_raster = arcpy.sa.Int(outName2 + "UrbRes"); out_raster.save(outName2 + "UrbResint")

In [452]:
# export as .tif 
arcpy.management.CopyRaster(outName2 + "UrbResint", OutTif + outName2 + ".tif", '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

<Result 'D:\\SE_Blueprint_2023\\7_CombineZonation\\Z_West_Gulf_Coastal_Plain_2023.tif'>

In [453]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

76.80943465232849
